In [10]:
# -*- coding: utf-8 -*-
"""
Análise de Classificações de Sentimentos
Este script processa logs brutos e consolidados para análise inicial das classificações.
"""

import pandas as pd
import numpy as np
import os
import csv

def carregar_arquivo_seguro(file_path):
    """
    Carrega um arquivo CSV de forma segura, tratando aspas corretamente.
    """
    df = pd.read_csv(
        file_path,
        sep=',',
        quoting=csv.QUOTE_ALL,
        quotechar='"',
        skipinitialspace=True,
        encoding='utf-8'
    )
    
    df.columns = df.columns.str.strip()
    df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
    
    return df

def process_log_file(file_path):
    """
    Processa um arquivo de log do PCIbex e extrai informações relevantes.
    """
    df = pd.read_csv(file_path, skiprows=19, header=None)
    
    df.columns = [
        "ReceptionTime", "ParticipantMD5", "Controller", "ItemNumber", "InnerElementNumber",
        "Label", "Group", "PennElementType", "PennElementName", "Parameter",
        "Value", "EventTime", "Comments"
    ]
    
    df = df[~df["ReceptionTime"].astype(str).str.startswith("#")].copy()
    df = df[~df["Label"].isin(["TCLE", "instrucoes", "agradecimento"])]
    df.reset_index(drop=True, inplace=True)
    
    # Extração de dados de gênero
    genero_data = df[
        (df["Label"] == "genero") &
        (df["PennElementName"] == "selecionaGenero") &
        (df["Parameter"] == "Selected")
    ][["ParticipantMD5", "Value"]].rename(columns={"Value": "Genero"}).drop_duplicates()
    
    # Processamento das classificações
    selecoes_frases = df[
        (df["Label"] == "frases") &
        (df["Parameter"] == "Selection")
    ].copy()
    
    selecoes_frases["ItemNumber"] = selecoes_frases["ItemNumber"].astype(int) - 3
    selecoes_frases = selecoes_frases[["ParticipantMD5", "ItemNumber", "Value", "EventTime"]]
    selecoes_frases.columns = ["ParticipantMD5", "ItemNumber", "Classificacao", "Timestamp"]
    
    return selecoes_frases, genero_data

def carregar_logs_brutos(log_folder, base_filename="results_prod", num_files=10):
    """
    Carrega e consolida todos os logs brutos disponíveis.
    """
    all_raw_classifications = []
    files_to_process = [f"{base_filename}.csv"] + [f"{base_filename} ({i}).csv" for i in range(1, num_files)]
    
    for filename in files_to_process:
        file_path = os.path.join(log_folder, filename)
        if not os.path.exists(file_path):
            continue
            
        print(f"Carregando classificações de: {filename}")
        
        try:
            frases, generos = process_log_file(file_path)
            generos["GeneroCod"] = generos["Genero"].str.lower().map(lambda g: "m" if "masculino" in g else "f")
            
            frases_com_genero = frases.merge(
                generos[["ParticipantMD5", "GeneroCod"]], 
                on="ParticipantMD5", 
                how="left"
            )
            all_raw_classifications.append(frases_com_genero)
            
        except Exception as e:
            print(f"Erro ao processar {filename}: {str(e)}")
    
    if all_raw_classifications:
        return pd.concat(all_raw_classifications, ignore_index=True)
    else:
        return pd.DataFrame()

def main():
    """
    Função principal que orquestra todo o processamento.
    """
    # Definição de caminhos
    log_folder = "logs/logs_brutos"
    log_processado_path = "dados/MQD_1465_log_processado.csv"
    output_consolidado_path = "dados/MQD_1465_classificacoes_consolidadas.csv"
    
    # Etapa 1: Carregar arquivo de logs processados
    print("Etapa 1: Carregando arquivo de logs processados...")
    df_processado = carregar_arquivo_seguro(log_processado_path)
    print(f"Carregados {len(df_processado)} registros do arquivo de logs processados.")
    
    # Etapa 2: Carregar e consolidar logs brutos
    print("\nEtapa 2: Carregando e consolidando logs brutos...")
    todas_classificacoes = carregar_logs_brutos(log_folder)
    print(f"Carregadas {len(todas_classificacoes)} classificações dos logs brutos.")
    
    # Etapa 3: Salvar consolidação
    print("\nEtapa 3: Salvando dados consolidados...")
    todas_classificacoes.to_csv(output_consolidado_path, index=False)
    print(f"Dados consolidados salvos em: {output_consolidado_path}")
    
    # Estatísticas básicas
    print("\nEstatísticas básicas:")
    print(f"Total de frases classificadas: {len(todas_classificacoes['ItemNumber'].unique())}")
    print("\nClassificações por gênero:")
    print(todas_classificacoes.groupby('GeneroCod')['ItemNumber'].count())

if __name__ == "__main__":
    main()

Etapa 1: Carregando arquivo de logs processados...
Carregados 1465 registros do arquivo de logs processados.

Etapa 2: Carregando e consolidando logs brutos...
Carregando classificações de: results_prod.csv
Carregando classificações de: results_prod (1).csv
Carregando classificações de: results_prod (2).csv
Carregando classificações de: results_prod (3).csv
Carregando classificações de: results_prod (4).csv
Carregando classificações de: results_prod (5).csv
Carregando classificações de: results_prod (6).csv
Carregando classificações de: results_prod (7).csv
Carregando classificações de: results_prod (8).csv
Carregando classificações de: results_prod (9).csv
Carregadas 13485 classificações dos logs brutos.

Etapa 3: Salvando dados consolidados...
Dados consolidados salvos em: dados/MQD_1465_classificacoes_consolidadas.csv

Estatísticas básicas:
Total de frases classificadas: 150

Classificações por gênero:
GeneroCod
f    6460
m    7025
Name: ItemNumber, dtype: int64
